<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/Ensemble%20Algorithms%20part%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

catBoost

In [ ]:
!pip install catboost

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool

df = pd.read_csv('/content/drive/MyDrive/Data.csv')

print(df.columns)

X_train, X_test, y_train, y_test = train_test_split(df.drop('Age', axis=1), df['Age'], test_size=0.2, random_state=42)

model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, loss_function='RMSE', task_type='GPU')

cat_features = ['Country']

train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

model.fit(train_pool, eval_set=test_pool, use_best_model=True, verbose=True)

y_pred = model.predict(test_pool)

mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse:.3f}')

Random subspace

In [11]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X, y = make_regression(n_samples=1000, n_features=20, n_informative=10, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class RandomSubspaceEnsembleRegressor:
    def __init__(self, base_estimator=RandomForestRegressor, n_estimators=10, max_features=0.5, random_state=42):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.random_state = random_state
        self.estimators_ = []

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            subset_features = np.random.choice(X.shape[1], int(X.shape[1] * self.max_features), replace=False)
            X_subset = X[:, subset_features]

            estimator = self.base_estimator(random_state=self.random_state)
            estimator.fit(X_subset, y)
            self.estimators_.append(estimator)

    def predict(self, X):
        predictions = []
        for estimator in self.estimators_:
            subset_features = np.random.choice(X.shape[1], int(X.shape[1] * self.max_features), replace=False)
            X_subset = X[:, subset_features]
            predictions.append(estimator.predict(X_subset))

        return np.mean(predictions, axis=0)

model = RandomSubspaceEnsembleRegressor(n_estimators=10, max_features=0.5, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.3f}')

Mean Squared Error: 29241.754


Stacking

In [12]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

X, y = make_regression(n_samples=1000, n_features=20, n_informative=10, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

base_models = [
    ('lr', LinearRegression()),
    ('knn', KNeighborsRegressor()),
    ('cart', DecisionTreeRegressor()),
    ('svm', SVR())
]

meta_model = LinearRegression()

model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.3f}')

Mean Squared Error: 0.000


voting clasifier

In [14]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
X = iris.data[:, :2]
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

base_models = [
    ('lr', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('cart', DecisionTreeClassifier())
]

model = VotingClassifier(estimators=base_models, voting='hard')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.900
